# Hotel Booking Demand Data Analysis

It is vacation time! And today we are using SQL on our “Hotel Booking Demand” dataset, to find some characteristics in the booking demand of two types of hotels. We will be comparing the guests’ choices of a city hotel and a resort hotel and answer the following questions:

- What is the high season for each hotel?
- What is the monthly average number of guests?
- What is the best time of the year to book a hotel room?

Finally, we will create visualizations to show our findings.

## 1\. Exploring our data.

First, let's take a glance at our dataset and see what kind of information it has available to us. Our database is relatively large, with 119.390 rows, so we will be just showing the 5 first results for each column.

In [1]:
-- Selecting top 5 rows of all columns
SELECT TOP(5) *
FROM Projects.dbo.hotel_bookings

(5 rows affected)

Total execution time: 00:00:00.019

hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
Resort Hotel,0,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0,0,0,Check-Out,7/1/2015
Resort Hotel,0,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0,0,0,Check-Out,7/1/2015
Resort Hotel,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75,0,0,Check-Out,7/2/2015
Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75,0,0,Check-Out,7/2/2015
Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,98,0,1,Check-Out,7/3/2015


- Refer to the Data Dictionary for more detailed information about the dataset.

Now that we have an idea of the content of the database, we can start our exploratory analysis. 

## 2\. What is the high season for each hotel?

To find the high season for each hotel, we will need to analyze the number of reservations for each month according to each hotel. Since this database contains data from July 2015 to August 2017, we will use the overall number of reservations through the years to find when is the high season. 

Let's select the number of reservations for both hotel types.

In [10]:
--Selecting the hotel type, month and number of reservations
--Grouped by the hotel type, month, and number of reservations
--City Hotel query:
SELECT hotel, arrival_date_month, COUNT(*) as reservations 
FROM [dbo].[hotel_bookings]
WHERE hotel = 'City Hotel'
GROUP BY hotel, arrival_date_month
ORDER BY hotel, reservations DESC 
--Resort Hotel query:
SELECT hotel, arrival_date_month, COUNT(*) as reservations 
FROM [dbo].[hotel_bookings]
WHERE hotel = 'Resort Hotel'
GROUP BY hotel, arrival_date_month
ORDER BY hotel, reservations DESC 

(12 rows affected)

(12 rows affected)

Total execution time: 00:00:00.205

hotel,arrival_date_month,reservations
City Hotel,August,8983
City Hotel,May,8232
City Hotel,July,8088
City Hotel,June,7894
City Hotel,October,7605
City Hotel,April,7480
City Hotel,September,7400
City Hotel,March,6458
City Hotel,February,4965
City Hotel,November,4357


hotel,arrival_date_month,reservations
Resort Hotel,August,4894
Resort Hotel,July,4573
Resort Hotel,April,3609
Resort Hotel,May,3559
Resort Hotel,October,3555
Resort Hotel,March,3336
Resort Hotel,September,3108
Resort Hotel,February,3103
Resort Hotel,June,3045
Resort Hotel,December,2648


It seems that August is the high season for each City Hotel and Resort Hotel. We can also notice a high demand in other months like May and July, so we can tell the high season is during summer.

## 3\. What is the monthly average number of guests?

In our database, we have the number of adults, children, and babies of each reservation. To find the average number of guests, we will add the number of people of each reservation and divide it by the number of months. Since our dataset it is from July 2015 to August 2017, we will be using the year 2016 to find the average number of guests.

In [8]:
--City Hotel query:
SELECT hotel, arrival_date_month ,(SUM(adults)+SUM(children)+SUM(babies))/12 as monthly_average
FROM [dbo].[hotel_bookings]
WHERE arrival_date_year = 2016 AND hotel = 'City Hotel'
GROUP BY hotel, arrival_date_month
ORDER BY monthly_average DESC
--Resort Hotel query:
SELECT hotel, arrival_date_month ,(SUM(adults)+SUM(children)+SUM(babies))/12 as monthly_average
FROM [dbo].[hotel_bookings]
WHERE arrival_date_year = 2016 AND hotel = 'Resort Hotel'
GROUP BY hotel, arrival_date_month
ORDER BY monthly_average DESC

(12 rows affected)

(12 rows affected)

Total execution time: 00:00:00.149

hotel,arrival_date_month,monthly_average
City Hotel,October,675
City Hotel,August,638
City Hotel,September,631
City Hotel,June,604
City Hotel,April,585
City Hotel,May,577
City Hotel,July,571
City Hotel,March,491
City Hotel,November,464
City Hotel,December,424


hotel,arrival_date_month,monthly_average
Resort Hotel,August,328
Resort Hotel,October,321
Resort Hotel,May,290
Resort Hotel,April,287
Resort Hotel,March,280
Resort Hotel,July,270
Resort Hotel,September,255
Resort Hotel,February,240
Resort Hotel,June,237
Resort Hotel,December,219


To verify that our average number of guests is accurate, we will compare the results of July 2016 with the other years to see if the numbers are close.

In [20]:
SELECT hotel,  arrival_date_year, (SUM(adults)+SUM(children)+SUM(babies))/12 as average
FROM [dbo].[hotel_bookings]
WHERE arrival_date_month = 'July'
GROUP BY hotel, arrival_date_year
ORDER BY hotel, arrival_date_year

(6 rows affected)

Total execution time: 00:00:00.131

hotel,arrival_date_year,average
City Hotel,2015,223
City Hotel,2016,571
City Hotel,2017,638
Resort Hotel,2015,252
Resort Hotel,2016,270
Resort Hotel,2017,344


We notice that the average number through the years is not too different, in fact, the number seems to be slowly increasing. There is also a noticeable difference between the average of July 2015 for City Hotel compared to the other years. As we know, our dataset started in July 2015 and we already determined that summer is the high season for City Hotel. Since we are missing data from before July, that may be the cause of the average number being skewed for the year 2015.

## 4\. What is the best time of the year to book a hotel room?

According to our table from question 3, we see that the lowest average of guests is either at the beginning of the year or at the end of the year. This may have an impact on our hotel rates and be the best time to book a hotel room. To confirm this hypothesis, let's take a look at the Average Daily Rate provided in our database to find the best time to make reservations.

To find the best rate, we will first find the mean value of the Average Daily Rate (ADR) for each hotel and look for dates that have an ADR below the mean value.

In [25]:
SELECT hotel, ROUND((SUM(adr) / COUNT(*)), 2) as mean_value
FROM [dbo].[hotel_bookings]
GROUP BY hotel

(2 rows affected)

Total execution time: 00:00:00.101

hotel,mean_value
City Hotel,105.3
Resort Hotel,94.95


Now that we know our mean value, we could look for dates when the ADRs are below 105.3 for City Hotel and 94.95 for Resort Hotel. But something is not quite right. When looking at our data, we see negative and zero values.

In [11]:
SELECT TOP(10) hotel, arrival_date_month,arrival_date_year, adr
FROM [dbo].[hotel_bookings]
WHERE adr < 105.3 
ORDER BY adr

(10 rows affected)

Total execution time: 00:00:00.084

hotel,arrival_date_month,arrival_date_year,adr
Resort Hotel,March,2017,-6.38
Resort Hotel,July,2015,0
Resort Hotel,July,2015,0
Resort Hotel,July,2015,0
Resort Hotel,July,2015,0
Resort Hotel,July,2015,0
Resort Hotel,July,2015,0
Resort Hotel,July,2015,0
City Hotel,January,2017,0
City Hotel,January,2017,0


In this case, we will treat those values as outliers and establish a minimal rate that is most likely to be the lowest ADR. We can establish the interquartile range (IQR) as a minimal value for our ADR and treat any value outside this range as an outlier. To do that, let's calculate the first and third quartile of both City Hotel and Resort Hotel.

In [14]:
--City Hotel query
SELECT
	hotel, adr_quartile,
	MAX(adr) AS quartile_break
FROM(
	SELECT
    	hotel,
		adr,
		NTILE(4) OVER (ORDER BY adr) AS adr_quartile
	FROM [dbo].[hotel_bookings]
	WHERE hotel = 'City Hotel') AS quartiles
WHERE adr_quartile IN (1, 3)
GROUP BY hotel, adr_quartile
--Resort Hotel query
SELECT
	hotel, adr_quartile,
	MAX(adr) AS quartile_break
FROM(
	SELECT
    	hotel,
		adr,
		NTILE(4) OVER (ORDER BY adr) AS adr_quartile
	FROM [dbo].[hotel_bookings]
	WHERE hotel = 'Resort Hotel') AS quartiles
WHERE adr_quartile IN (1, 3)
GROUP BY hotel, adr_quartile

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.515

hotel,adr_quartile,quartile_break
City Hotel,1,79.2
City Hotel,3,126


hotel,adr_quartile,quartile_break
Resort Hotel,1,50
Resort Hotel,3,125


Now that we have our quartiles, we calculate the IQR and find the lowest ADR. In this case, the lowest ADR of City Hotel would be 46.8 and 75 for Resort Hotel. We can find now when is the best time to book a hotel room according to the lowest Average Daily Rate. Since our database has daily rates of three years, we will calculate the total average of those years to find the best month to make reservations.

In [13]:
--City Hotel query
SELECT hotel, arrival_date_month,ROUND(AVG( adr),2) as total_avg
FROM [dbo].[hotel_bookings]
WHERE adr > 46.8 and hotel = 'City Hotel' 
GROUP BY hotel, arrival_date_month
ORDER BY total_avg
--Resort Hotel query
SELECT hotel, arrival_date_month,ROUND(AVG( adr),2) as total_avg
FROM [dbo].[hotel_bookings]
WHERE adr > 75 and hotel = 'Resort Hotel' 
GROUP BY hotel, arrival_date_month
ORDER BY total_avg

(12 rows affected)

(12 rows affected)

Total execution time: 00:00:00.228

hotel,arrival_date_month,total_avg
City Hotel,January,84.92
City Hotel,February,87.49
City Hotel,November,90.61
City Hotel,December,92.14
City Hotel,March,94.64
City Hotel,October,102.2
City Hotel,September,112.11
City Hotel,July,112.54
City Hotel,April,112.82
City Hotel,August,116.75


hotel,arrival_date_month,total_avg
Resort Hotel,November,90.53
Resort Hotel,January,90.6
Resort Hotel,February,91.72
Resort Hotel,March,94.12
Resort Hotel,October,98.57
Resort Hotel,April,101.31
Resort Hotel,May,104.93
Resort Hotel,September,116.42
Resort Hotel,June,126
Resort Hotel,December,126.38


<span style="font-size: 10.5pt; line-height: 107%; font-family: &quot;Segoe UI&quot;, sans-serif; background-image: initial; background-position: initial; background-size: initial; background-repeat: initial; background-attachment: initial; background-origin: initial; background-clip: initial;">It seems that January, February, and November have the lowest daily average rate for either City Hotel or Resort Hotel. This makes sense once those months were at the bottom of our monthly average guests' table and at the bottom of our high season table.&nbsp;</span> 

## 5\. Conclusion

In this analysis, we explored the Hotel Booking dataset that contains over 100,000 rows of data using T-SQL and Microsoft SQL Server Management Studio. We started performing initial exploration of the data, such as querying the first rows and operating basic data cleaning (changing columns data type) as needed. Once the dataset was ready, we started our exploratory data analysis to find the high season of hotel booking demand and when is the best time to make hotel reservations.

After querying our database to count how many reservations were made and grouping into months, we found that the highest season of City Hotel and Resort Hotel was during summer. We can confirm this analysis when calculating the monthly average number of guests and finding that the average is higher from May to October. Lastly, we used the Average Daily Rate (ADR) to find when it is the best time to make hotel reservations. We used statistics fundamentals to exclude outliers and calculated the total ADR average to find the month with the lowest rate. We concluded that January, February, and November have the lowest average rates hence it is the best time to book a hotel room.